# Music Taste Prediction Model: New Music Friday Recommender 💿🎧👍👎
In this model, I use my liked songs playlist, my recently loved and not loved albums, to train my regression model on what kind of music I do and don't like. At the end my test model will be the new music friday albums from the most recent Friday. 

# Library Imports

In [2]:
# Standard Library Imports
import os
import csv
from datetime import datetime
from time import sleep
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Tuple, Optional

# Third-Party Imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from textblob import TextBlob
from transformers import pipeline
from collections import Counter

# Fuzzy Matching
from fuzzywuzzy import fuzz, process

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, make_scorer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from scipy.stats import randint, uniform

# Network Analysis
import networkx as nx

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm
from nbconvert import HTMLExporter
import nbformat

# Streamlit 
import streamlit as st

# Suppress all warnings
import warnings
warnings.filterwarnings('ignore')


# Load Datasets

In [ ]:
df_liked = pd.read_csv("data/liked.csv")  # Liked playlist on Spotify
df_fav_albums = pd.read_csv("data/liked_albums.csv")  # Albums I've Liked in Recent Years
df_not_liked = pd.read_csv("data/did_not_like.csv")  # Albums I've not liked in Recent Years
df_nmf = pd.read_csv("data/nmf.csv")  # The most recent New Music Friday Playlist
df_mid = pd.read_csv("data/mid.csv") #Albums I neither enjoyed or hated, but were simply Mid
df_liked_similar = pd.read_csv('data/liked_artists_only_similar.csv') #Similar Artisits to My Liked Artists

## A Check for New Artists / Pull Their Similar Artists

In [ ]:
# Initialize LastFM API client with rate limiting and request limits
# This ensures we don't overload the API and stay within usage guidelines ⏳
class LastFMAPI:
    def __init__(self, api_key: str, rate_limit_delay: float = 0.25, limit: int = 8):
        self.api_key = api_key
        self.base_url = "http://ws.audioscrobbler.com/2.0/"
        self.rate_limit_delay = rate_limit_delay
        self.limit = limit

    # Fetch similar artists for a given artist from LastFM API
    # This is the core function that interacts with the API 🎤
    def get_similar_artists(self, artist_name: str) -> List[str]:
        params = {
            'method': 'artist.getSimilar',
            'artist': artist_name,
            'api_key': self.api_key,
            'limit': self.limit,  # Limit the number of similar artists returned 🎯
            'format': 'json'
        }
        
        try:
            response = requests.get(self.base_url, params=params)
            response.raise_for_status()
            
            # Handle rate limiting to avoid API bans ⚠️
            if 'X-RateLimit-Remaining' in response.headers:
                remaining = int(response.headers['X-RateLimit-Remaining'])
                if remaining == 0:
                    sleep(self.rate_limit_delay)
            
            # Extract and return similar artists from the API response 🎶
            data = response.json()
            if 'similarartists' in data and 'artist' in data['similarartists']:
                return [artist['name'] for artist in data['similarartists']['artist'][:self.limit]]
            return []
            
        except Exception as e:
            print(f"Error fetching similar artists for {artist_name}: {e}")
            return []

# Extract the primary artist name from a string
# This handles cases where multiple artists are listed (e.g., "Artist A, Artist B") 🎤
def extract_primary_artist(artist_string: str) -> str:
    if pd.isna(artist_string):
        return ""
    return artist_string.split(",")[0].strip()

# Update the similar artists database with new artists from liked playlists
# This function ensures the database stays up-to-date with my latest music preferences 🔄
def update_similar_artists(liked_path: str, 
                         albums_path: str, 
                         output_path: str, 
                         api_key: str) -> pd.DataFrame:
    print("Loading existing and new data...")
    
    # Load existing similar artists data
    # This ensures we don't duplicate work for artists already in the database 📂
    existing_data: Dict[str, List[str]] = {}
    if os.path.exists(output_path):
        existing_df = pd.read_csv(output_path)
        existing_data = dict(zip(existing_df['Artist'], existing_df['Similar Artists']))
        print(f"Loaded {len(existing_data)} existing artists from database")
    
    # Load and process current playlists
    # This combines liked songs and albums into a single set of artists 🎧
    df_liked = pd.read_csv(liked_path)
    df_albums = pd.read_csv(albums_path)
    
    # Extract and combine primary artists
    current_artists = set(
        pd.concat([
            df_liked['Artist Name(s)'].apply(extract_primary_artist),
            df_albums['Artist Name(s)'].apply(extract_primary_artist)
        ]).unique()
    )
    current_artists.discard("")  # Remove empty strings
    
    # Find new artists not in existing data
    # This ensures we only process artists we haven't seen before �
    new_artists = current_artists - set(existing_data.keys())
    print(f"Found {len(new_artists)} new artists to process")
    
    if not new_artists:
        print("No new artists to process. Database is up to date!")
        # Create and return DataFrame even if no updates
        return pd.DataFrame({
            'Artist': list(existing_data.keys()),
            'Similar Artists': list(existing_data.values())
        })
    
    # Initialize LastFM API client
    # This is where the magic happens ✨
    api = LastFMAPI(api_key)
    
    # Process artists with concurrent requests
    # This speeds up the process by fetching data in parallel ⚡
    results = {}
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_artist = {
            executor.submit(api.get_similar_artists, artist): artist 
            for artist in new_artists
        }
        
        # Show progress bar while processing
        # This keeps you informed about the progress of the task 📊
        for future in tqdm(as_completed(future_to_artist), 
                         total=len(future_to_artist),
                         desc="Fetching similar artists"):
            artist = future_to_artist[future]
            similar_artists = future.result()
            results[artist] = ', '.join(similar_artists)
    
    # Combine existing and new data
    # This ensures the final dataset includes all artists, old and new 🔗
    combined_data = {**existing_data, **results}
    
    # Create DataFrame
    # This formats the data for easy saving and future use 📄
    output_df = pd.DataFrame({
        'Artist': list(combined_data.keys()),
        'Similar Artists': list(combined_data.values())
    })
    
    # Save updated data
    # This ensures the database is persisted for future runs 💾
    output_df.to_csv(output_path, index=False)
    print(f"Successfully updated database with {len(new_artists)} new artists")
    print(f"Total artists in database: {len(combined_data)}")
    
    return output_df

# Main execution block
# This runs the update process when the script is executed directly 🚀
if __name__ == "__main__":
    # Configuration
    API_KEY = "74a510ecc9fc62bf3e0edc6adc2e99f9"
    LIKED_PATH = "data/liked.csv"
    ALBUMS_PATH = "data/liked_albums.csv"
    OUTPUT_PATH = "data/liked_artists_only_similar.csv"
    
    # Run the update and get the DataFrame
    df_liked_similar = update_similar_artists(
        LIKED_PATH, 
        ALBUMS_PATH, 
        OUTPUT_PATH, 
        API_KEY
    )
    
# Now df_liked_similar is ready to use
df_liked_similar.head()

## Quick Glance at our Refreshed Datasets

In [ ]:
df_liked.head()

In [ ]:
# Liked Albums in Recent Years
df_fav_albums.head()

In [ ]:
# Albums Not Liked in Recent Years
df_not_liked.head()

In [ ]:
#Albums I neither enjoyed or hated, but were simply Mid
df_mid.head()

In [ ]:
# New Music Friday Playlist
df_nmf.head()

In [ ]:
# Similar Artists to Recently Played Artists (Last.fm)

df_liked_similar.head()

> A quick reminder of the standard columns of a spotify export.

In [ ]:
df_liked.columns

> What's available in the Similar dataframe?

In [ ]:
df_liked_similar.columns

### Add Target Labels for Training Feature
We need to assign a score to songs I've faved on spotify (100), albums I've enjoyed in recent years (65), albums that were mid (45), and albums that I have not enjoyed in recent years (30) to train the model on the types of songs I don't like, like, and love. 'liked' will be our target variable, later.

In [ ]:
# Assign liked scores before combining
df_liked['liked'] = 100
df_fav_albums['liked'] = 65
df_mid['liked'] = 45
df_not_liked['liked'] = 30
df_nmf['liked'] = np.nan 

# Add playlist_origin column before combining
df_liked['playlist_origin'] = 'df_liked'
df_fav_albums['playlist_origin'] = 'df_fav_albums'
df_not_liked['playlist_origin'] = 'df_not_liked'
df_mid['playlist_origin'] = 'df_mid'
df_nmf['playlist_origin'] = 'df_nmf'
df_liked_similar['source'] = 'liked_similar'

### Check application of the target encoding

In [ ]:
df_liked[['liked', 'playlist_origin']].head()

In [ ]:
df_fav_albums[['liked', 'playlist_origin']].head()

In [ ]:
df_mid[['liked', 'playlist_origin']].head()

In [ ]:
df_not_liked[['liked', 'playlist_origin']].head()

In [ ]:
df_nmf[['liked', 'playlist_origin']].head()

In [ ]:
df_liked_similar[['Artist', 'Similar Artists', 'source']].head()

## Data Cleaning 🧹
Before diving into modeling, I need to clean and prepare the data to ensure it’s ready for analysis. This section focuses on merging my datasets, handling duplicates, filling missing values, and making sure the data is in the best possible shape. Here’s what I’ll tackle:

1. **Merging Datasets**: I’ll combine my liked songs, favorite albums, not-liked albums, and New Music Friday tracks into a single dataframe.
2. **Removing Duplicates**: I’ll ensure each track is unique, prioritizing higher "liked" scores for duplicates.
3. **Handling Missing Values**: I’ll fill gaps in genres using Last.fm’s richer data and save a backup of missing artists for manual review later.
4. **Dropping Irrelevant Columns**: I’ll remove columns that won’t contribute to the model.
5. **Final Checks**: I’ll verify the dataset’s integrity and distribution before moving to modeling.

By the end of this section, I’ll have a clean, unified dataset ready for feature engineering and model training. Let’s get started! 🚀

## Merge The Datasets

In [ ]:
df = pd.concat([df_liked, df_fav_albums, df_mid, df_not_liked, df_nmf], ignore_index=True)

In [ ]:
#How Large is the Dataset, Now?
df.shape

#### Remove the Duplicates 🔥

In [ ]:
# Remove duplicates: Keep the highest 'liked' score (100 > 65)
df = df.sort_values(by='liked', ascending=False)  # Ensures 100-rated songs come first
df = df.drop_duplicates(subset=['Track Name', 'Artist Name(s)'], keep='first')
df.shape

#### Drop columns that won't help the model 💣

In [ ]:
df.drop(columns=['Added By', 'Added At', 'Time Signature'], inplace=True)

#### Handle missing values (if any)

In [ ]:
 df.isna().sum()

## Getting New Genre Data From Last.fm 🟢🎵
Since half of the genres are missing from Spotify songs, I decided to use the richer data from Last.fm's crowd-sourced genre tags. There are some silly tags in there, so I added a list of ignored tags that I update weekly based on what I see in the data. This ensures the genres are meaningful and relevant for my model.

In [ ]:
class LastFMGenreFetcher:
    def __init__(self, api_key: str, rate_limit_delay: float = 0.25):
        self.api_key = api_key
        self.base_url = "http://ws.audioscrobbler.com/2.0/"
        self.rate_limit_delay = rate_limit_delay
        
        # Define tags to ignore
        # These are the crowd-sourced tags that are more about personal taste than actual genres
        self.ignored_tags = {
            'seen live',
            'albums i own',
            'favorite',
            'favourites',
            'favourite',
            'my playlist',
            'spotify',
            'pandora',
            'wish i had seen live',
            'awesome',
            'love at first listen',
            'love',
            'amazing',
            'listened',
            'personal',
            'my music'
        }
        
        # Define variations of female vocalist to standardize
        # Because Last.fm users love to tag the same thing in 10 different ways 🎤
        self.female_vocalist_variants = {
            'female vocalists',
            'female vocalist',
            'female vocals',
            'female fronted',
            'female voices',
            'female voice',
            'female singers',
            'female singer'
        }
        
        # Genre groups that typically appear together
        self.genre_compatibility = {
            'indie': ['indie pop', 'indie rock', 'indie folk', 'alternative', 'singer-songwriter'],
            'rock': ['alternative rock', 'indie rock', 'hard rock', 'classic rock', 'punk'],
            'electronic': ['techno', 'house', 'edm', 'ambient', 'idm', 'electronica'],
            'metal': ['heavy metal', 'death metal', 'black metal', 'thrash metal', 'doom metal'],
            'folk': ['indie folk', 'folk rock', 'americana', 'singer-songwriter', 'acoustic'],
            'pop': ['indie pop', 'synth pop', 'dream pop', 'electropop', 'pop rock'],
            'hip-hop': ['rap', 'trap', 'r&b', 'urban', 'grime'],
            'jazz': ['fusion', 'bebop', 'smooth jazz', 'soul', 'funk'],
            'classical': ['orchestral', 'chamber music', 'piano', 'instrumental', 'contemporary classical']
        }
        
        # This identifies genres that rarely appear together
        self.genre_incompatibility = {
            'classical': ['metal', 'rap', 'edm', 'techno', 'dubstep'],
            'death metal': ['pop', 'r&b', 'jazz', 'folk', 'ambient'],
            'christian': ['black metal', 'satanic', 'pagan'],
            'country': ['techno', 'edm', 'black metal', 'death metal'],
            'jazz': ['black metal', 'death metal', 'hardcore', 'screamo'],
            'k-pop': ['death metal', 'black metal', 'doom metal']
        }
        
    def _make_request(self, params: Dict)  -> Dict:
        try:
            response = requests.get(self.base_url, params=params)
            response.raise_for_status()
            sleep(self.rate_limit_delay)  # Basic rate limiting to avoid angering the Last.fm API gods
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"API request failed for params {params}: {e}")
            if getattr(response, 'status_code', None) == 429:
                print("Rate limit exceeded, increasing delay. Patience is a virtue!")
                self.rate_limit_delay *= 2
                sleep(5)  # Wait before retry
            return None

    def get_artist_tags(self, artist_name: str) -> List[str]:
        params = {
            'method': 'artist.getTopTags',
            'artist': artist_name,
            'api_key': self.api_key,
            'format': 'json'
        }
        data = self._make_request(params)
        
        if not data or 'toptags' not in data:
            return []  # Sometimes Last.fm just doesn't have the data 🤷‍♂️
            
        tags = []
        seen_tags = set()  # Track unique tags to avoid duplicates
        has_female_vocalist = False
        
        for tag in data['toptags'].get('tag', []):
            tag_name = tag['name'].lower()
            
            # Skip if tag is in ignored list
            if tag_name in self.ignored_tags:
                continue  # Bye-bye, irrelevant tags!
                
            # Handle female vocalist variations
            if tag_name in self.female_vocalist_variants:
                if not has_female_vocalist:
                    tags.insert(0, 'female vocalist')  # Standardize and prioritize this tag
                    has_female_vocalist = True
                continue
            
            # Only add tag if we haven't seen it before
            if tag_name not in seen_tags:
                tags.append(tag_name)
                seen_tags.add(tag_name)
                
        return tags[:10]  # Return up to 10 tags because less is more, right?
        
    def get_album_tags(self, artist_name: str, album_name: str, release_date: str = None) -> List[str]:
        """
        Get tags specifically for an album, with verification against artist tags.
        """
        # Try to get album-specific tags first
        params = {
            'method': 'album.getTopTags',
            'artist': artist_name,
            'album': album_name,
            'api_key': self.api_key,
            'format': 'json'
        }
        album_data = self._make_request(params)
        
        # Get artist tags for comparison
        artist_tags = self.get_artist_tags(artist_name)
        
        # Extract album tags if available
        album_tags = []
        if album_data and 'toptags' in album_data and 'tag' in album_data['toptags']:
            for tag in album_data['toptags'].get('tag', []):
                tag_name = tag['name'].lower()
                if tag_name not in self.ignored_tags:
                    album_tags.append(tag_name)
        
        # This verifies tags when we have both album and artist data
        if album_tags and artist_tags:
            verified_tags = self._verify_tags(album_tags, artist_tags, release_date)
            return verified_tags[:10]
        elif album_tags:
            return album_tags[:10]
        else:
            return artist_tags[:10]
    
    def _verify_tags(self, album_tags: List[str], artist_tags: List[str], release_date: str = None) -> List[str]:
        # Convert to sets for easier comparison
        album_tag_set = set(album_tags)
        artist_tag_set = set(artist_tags)
        
        # Calculate overlap between tags
        overlap = album_tag_set.intersection(artist_tag_set)
        overlap_ratio = len(overlap) / len(album_tag_set) if album_tag_set else 0
        
        # This means we have good tag agreement
        if overlap_ratio >= 0.3:
            return album_tags
        
        # Check for incompatible genres
        for tag in album_tags:
            for incompatible_genre, incompatible_tags in self.genre_incompatibility.items():
                if tag == incompatible_genre:
                    for artist_tag in artist_tags:
                        if artist_tag in incompatible_tags:
                            # This indicates a likely mismatch
                            return artist_tags
        
        # Use release date as tiebreaker if available
        if release_date:
            try:
                year = int(release_date.split('-')[0])
                current_year = datetime.now().year
                
                # This prioritizes album tags for recent releases
                if year >= current_year - 2:
                    merged_tags = album_tags[:7] + [tag for tag in artist_tags if tag not in album_tag_set][:3]
                    return merged_tags
            except:
                pass
        
        # Merge tags with preference to album tags
        merged_tags = album_tags[:7] + [tag for tag in artist_tags if tag not in album_tag_set][:3]
        return merged_tags

def update_genre_data(api_key: str, dataframes: List[pd.DataFrame], output_file: str = 'data/ten_genres.csv') -> None:
    api = LastFMGenreFetcher(api_key)
    
    # Cache for artist tags
    artist_tags_cache = {}

    def get_album_tags_cached(artist_name, album_name, release_date=None):
        if artist_name not in artist_tags_cache:
            artist_tags_cache[artist_name] = api.get_artist_tags(artist_name)
        artist_tags = artist_tags_cache[artist_name]

        params = {
            'method': 'album.getTopTags',
            'artist': artist_name,
            'album': album_name,
            'api_key': api.api_key,
            'format': 'json'
        }
        album_data = api._make_request(params)
        
        album_tags = []
        if album_data and 'toptags' in album_data and 'tag' in album_data['toptags']:
            for tag in album_data['toptags']['tag']:
                tag_name = tag['name'].lower()
                if tag_name not in api.ignored_tags:
                    album_tags.append(tag_name)
        
        if album_tags:
            return api._verify_tags(album_tags, artist_tags, release_date)[:10]
        else:
            return artist_tags[:10]

    # Gather all unique artist-album pairs
    artist_album_pairs = []
    for df in dataframes:
        if 'Artist Name(s)' in df.columns and 'Album Name' in df.columns and 'Release Date' in df.columns:
            pairs = df[['Artist Name(s)', 'Album Name', 'Release Date']].drop_duplicates()
            for _, row in pairs.iterrows():
                artist = row['Artist Name(s)'].split(',')[0].strip() if pd.notna(row['Artist Name(s)']) else None
                album = row['Album Name'] if pd.notna(row['Album Name']) else None
                release_date = row['Release Date'] if pd.notna(row['Release Date']) else None
                if artist and album:
                    artist_album_pairs.append((artist, album, release_date))

    # Load existing data
    existing_data = {}
    if os.path.exists(output_file) and os.path.getsize(output_file) > 0:
        try:
            existing_data = pd.read_csv(output_file).set_index(['Artist', 'Album'])['Genres'].to_dict()
        except (pd.errors.EmptyDataError, KeyError):
            print("No existing genre data found or file is empty. Starting fresh.")

    # Load obscure artist genres
    obscure_artists = pd.read_csv("data/obscure_artists_mike_likes.csv", quotechar='"')
    obscure_artists_dict = { (row['Artist'], 'ANY_ALBUM'): row['Genres'] for _, row in obscure_artists.iterrows() }
    existing_data.update(obscure_artists_dict)

    # Identify new artist-album pairs
    new_pairs = [(artist, album, release_date) for artist, album, release_date in artist_album_pairs]

    if not new_pairs:
        print("No new artist-album pairs to process!")
        return

    print(f"Found {len(new_pairs)} new artist-album pairs to process.")

    # Process new pairs in parallel
    results = {}
    with ThreadPoolExecutor(max_workers=25) as executor:
        future_to_pair = {
            executor.submit(get_album_tags_cached, artist, album, release_date): (artist, album)
            for artist, album, release_date in new_pairs
        }

        completed = 0
        for future in as_completed(future_to_pair):
            artist, album = future_to_pair[future]
            try:
                tags = future.result()
                if tags:
                    results[(artist, album)] = ', '.join(tags)
                completed += 1
                if completed % 50 == 0 or (len(new_pairs) < 50 and completed % 10 == 0):
                    print(f"Processed {completed}/{len(new_pairs)} artist-album pairs.")
            except Exception as e:
                print(f"Error processing {artist} - {album}: {e}")

    # Combine existing and new data
    all_data = {**existing_data, **results}

    # Save to CSV
    df_output = pd.DataFrame(
        [{'Artist': artist, 'Album': album, 'Genres': genres} for (artist, album), genres in all_data.items()]
    )
    df_output.to_csv(output_file, index=False)
    print(f"\nSuccessfully updated {output_file} with {len(results)} new artist-album pairs!")


# Define genre cleaning function
def clean_genres(genre_string):
    """
    Clean genre strings by:
    1. Removing duplicates
    2. Removing '|' bars
    3. Filtering out genres with numbers
    4. Filtering out genres longer than 3 words
    """
    if pd.isna(genre_string):
        return genre_string
        
    # Split by | and flatten all genres into one list
    all_parts = [part.strip() for part in genre_string.split('|')]
    all_genres = []
    for part in all_parts:
        genres = [g.strip() for g in part.split(',')]
        all_genres.extend(genres)
    
    # Remove duplicates, numbers, and long genres while preserving order
    seen = set()
    unique_genres = []
    for genre in all_genres:
        # Skip if empty
        if not genre:
            continue
        
        # Skip if contains digits
        if any(char.isdigit() for char in genre):
            continue
            
        # Skip if longer than 3 words
        if len(genre.split()) > 3:
            continue
            
        # Add if not seen before (case-insensitive check)
        if genre.lower() not in seen:
            unique_genres.append(genre)
            seen.add(genre.lower())
    
    return ', '.join(unique_genres)

# Example usage:
if __name__ == "__main__":
    API_KEY = '74a510ecc9fc62bf3e0edc6adc2e99f9'

    cache_file = "data/ten_genres.csv"
    obscure_file = "data/obscure_artists_mike_likes.csv"

    # Load cached ten_genres
    if os.path.exists(cache_file) and os.path.getsize(cache_file) > 0:
        cached_df = pd.read_csv(cache_file)
        cached_df['Primary Artist'] = cached_df['Artist'].str.split(',').str[0].str.strip()
        cached_pairs_set = set(zip(cached_df['Primary Artist'], cached_df['Album']))
        cached_any_album_set = set(cached_df['Primary Artist'])
    else:
        cached_pairs_set = set()
        cached_any_album_set = set()

    # Load obscure artists
    if os.path.exists(obscure_file):
        obscure_df = pd.read_csv(obscure_file, quotechar='"')
        obscure_df['Primary Artist'] = obscure_df['Artist'].str.split(',').str[0].str.strip()
        obscure_any_album_set = set(obscure_df['Primary Artist'])
    else:
        obscure_any_album_set = set()

    # Extract primary artist from main df
    df['Primary Artist'] = df['Artist Name(s)'].str.split(',').str[0].str.strip()

    # Identify truly new artist-album pairs
    df['Primary Artist'] = df['Artist Name(s)'].str.split(',').str[0].str.strip()

    # Vectorized filtering
    is_new_pair = ~df.apply(lambda r: (r['Primary Artist'], r['Album Name']) in cached_pairs_set, axis=1)
    is_not_in_cached_any = ~df['Primary Artist'].isin(cached_any_album_set)
    is_not_in_obscure = ~df['Primary Artist'].isin(obscure_any_album_set)

    df_new = df[is_new_pair & is_not_in_cached_any & is_not_in_obscure]

    print(f"Processing {len(df_new)} new artist-album pairs.")
    
    # Update genre data
    update_genre_data(API_KEY, [df_new], 'data/ten_genres.csv')

    # Load the ten_genres.csv file
    ten_genres = pd.read_csv("data/ten_genres.csv")
    
    # Clean genres in the ten_genres DataFrame
    ten_genres["Genres"] = ten_genres["Genres"].apply(clean_genres)
    
    # Save the cleaned ten_genres back to CSV
    ten_genres.to_csv("data/ten_genres.csv", index=False)

    # Extract the primary artist (before the comma)
    df["Primary Artist"] = df["Artist Name(s)"].str.split(",").str[0]
    ten_genres["Primary Artist"] = ten_genres["Artist"].str.split(",").str[0]

    # Merge, prioritizing ten_genres
    df = df.merge(ten_genres[["Primary Artist", "Genres"]], on="Primary Artist", how="left", suffixes=("", "_ten"))

    # If ten_genres has a match, use it; otherwise, keep the original
    df["Genres"] = df["Genres_ten"].combine_first(df["Genres"])

    # Apply the cleaning function to clean up genre data
    df["Genres"] = df["Genres"].apply(clean_genres)

    # Drop the extra columns
    df.drop(columns=["Genres_ten", "Primary Artist"], inplace=True)

### Save a Copy of the Missing Genre Artists (Just in Case I Want to Fuss With It Later!) 🕵️‍♂️🎶
Even with Last.fm’s extensive data, some artists still slip through the cracks. To handle these edge cases, I’m saving a list of artists with missing genre data to a separate file. This way, I can manually review and assign genres later if needed. Think of it as a backup plan for those hard-to-categorize artists! 🎧✨

In [ ]:
# Load the ten_genres.csv file
ten_genres = pd.read_csv("data/ten_genres.csv")

# Extract the primary artist from both datasets
# For artists with collaborators, we focus on the first name listed 🎤
df["Primary Artist"] = df["Artist Name(s)"].str.split(",").str[0].str.strip()
ten_genres["Primary Artist"] = ten_genres["Artist"].str.split(",").str[0].str.strip()

# Identify missing artists
# These are the artists not found in the ten_genres.csv file 🎸
missing_artists = df[~df["Primary Artist"].isin(ten_genres["Primary Artist"])]

# Save the missing artists to a CSV file
# This allows for manual review and genre assignment later 🕵️‍♂️
missing_artists[["Primary Artist"]].drop_duplicates().to_csv("data/missing_artists.csv", index=False)

print(f"Saved {len(missing_artists)} missing artists to 'data/missing_artists.csv'.")

# The missing artists are now stored in data/missing_artists.csv.
# If needed, I can manually assign genres and add them to
# obscure_artists_mike_likes.csv for future runs. A little manual effort goes a long way! 🎛️

In [ ]:
#How many songs still have missing genre data?
df.isna().sum()

In [ ]:
# Drop rows with nulls in any column except 'liked' because those belong to playlist_origin = df_nmf
df = df[df.drop(columns=['liked']).notna().all(axis=1)].reset_index(drop=True)

In [ ]:
 df.isna().sum()

In [ ]:
df.shape

In [ ]:
# Count how many tracks belong to each 'playlist_origin' in the dataset
# This helps us understand the distribution of tracks across different sources 🎵
playlist_origin_counts = df['playlist_origin'].value_counts()

# Print the results
print("Playlist Origin Counts:")
playlist_origin_counts

## Finding Top 30 Genres (Temporarily, for new feature)

In [ ]:
# Step 1: Explode the Genres column
df_exploded = df.assign(Genres=df['Genres'].str.split(', ')).explode('Genres')

# Step 2: Filter out genres that appear fewer than 10 times
genre_counts = df_exploded['Genres'].value_counts()
frequent_genres = genre_counts[genre_counts >= 40].index  # Genres with at least 10 occurrences

# Filter the exploded DataFrame to only include frequent genres
df_frequent_genres = df_exploded[df_exploded['Genres'].isin(frequent_genres)]

# Step 3: Group by Genre and calculate the mean of the 'liked' column
genre_liked_avg = df_frequent_genres.groupby('Genres')['liked'].mean().reset_index()

# Step 4: Sort by the average 'liked' score in descending order and get the top 30
top_30_genres = genre_liked_avg.sort_values(by='liked', ascending=False).head(30)

# Display the top 30 genres
print("Top 30 Genres by Average Liked Score (Filtered for Frequent Genres):")
print(top_30_genres)

## Feature Engineering / Further Selecting 👌

### 'Record Label Frequency Encoded' 🏷️

To improve the model’s ability to generalize, I’m creating a new feature: **Record Label Frequency Encoded**. This feature represents how frequently each record label appears in the dataset, but with a twist—we only consider labels from tracks I’ve liked or marked as favorite albums. This ensures the encoding reflects my preferences and ignores labels from tracks I don’t care about.

#### How It Works:
1. **Training Data Preparation**: We filter the combined dataframe (`df`) by `playlist_origin` to get subsets for `df_liked` and `df_fav_albums`. This ensures we’re working with the most up-to-date data.
2. **Frequency Calculation**: We count how often each record label appears in the training data (i.e., liked and favorite albums).
3. **Handling Unseen Labels**: For labels not present in the training data (e.g., those in the NMF test set), we use the **mean frequency** of all labels as a fallback. This ensures the model can handle new or rare labels gracefully.
4. **Application**: We apply the encoding to both the training and test sets, ensuring consistency across the data.

#### Why This Matters:
- **Relevance**: By focusing only on labels from tracks I like, the encoding better reflects my preferences.
- **Robustness**: Using the mean frequency for unseen labels ensures new artists from rare labels won't be penalized.
- **Simplicity**: Frequency encoding avoids the complexity of one-hot encoding while still capturing meaningful information.

This approach ensures the model learns from the distribution of labels I care about while remaining robust to new or rare labels.

In [ ]:
### 'Record Label Frequency Encoded' 🏷️

# Filter the combined dataframe by playlist_origin to get the training subset
# We focus on liked and favorite albums for training, ignoring disliked tracks and NMF 🚫
df_train_only = df[df['playlist_origin'].isin(['df_liked', 'df_fav_albums'])]

# Calculate frequency encoding using only liked and favorite albums
# This ensures the encoding reflects the distribution of labels I care about 🏷️
freq_encoding = df_train_only['Record Label'].value_counts()

# Get the mean frequency to use for unseen labels
# This acts as a fallback for labels not present in the training data 🛠️
mean_freq = freq_encoding.quantile(0.25)

# Apply encoding to the training data
df.loc[df['playlist_origin'].isin(['df_liked', 'df_fav_albums']), 'Record Label Frequency Encoded'] = (
    df_train_only['Record Label'].map(freq_encoding).fillna(mean_freq)
)

# Apply encoding to the NMF test data separately
# This avoids data leakage by using only the training data's frequency encoding 🚀
df_nmf_subset = df[df['playlist_origin'] == 'df_nmf']
df_nmf_subset['Record Label Frequency Encoded'] = df_nmf_subset['Record Label'].map(freq_encoding).fillna(mean_freq)
df.loc[df['playlist_origin'] == 'df_nmf', 'Record Label Frequency Encoded'] = df_nmf_subset['Record Label Frequency Encoded']

# Apply encoding to the disliked albums
df_not_liked_subset = df[df['playlist_origin'] == 'df_not_liked']
df_not_liked_subset['Record Label Frequency Encoded'] = df_not_liked_subset['Record Label'].map(freq_encoding).fillna(mean_freq)
df.loc[df['playlist_origin'] == 'df_not_liked', 'Record Label Frequency Encoded'] = df_not_liked_subset['Record Label Frequency Encoded']

# Apply encoding to the mid albums
df_mid_subset = df[df['playlist_origin'] == 'df_mid']
df_mid_subset['Record Label Frequency Encoded'] = df_mid_subset['Record Label'].map(freq_encoding).fillna(mean_freq)
df.loc[df['playlist_origin'] == 'df_mid', 'Record Label Frequency Encoded'] = df_mid_subset['Record Label Frequency Encoded']


# Check the result by displaying 20 Unique Record Labels and Their Encoding!
df[['Record Label', 'Record Label Frequency Encoded']].drop_duplicates().head(20)

### Target Encode Genres 🎚

To capture the relationship between genres and the target variable (liked), I use target encoding. This technique replaces each genre (or combination of genres) with the mean target value for that genre, smoothed to handle rare categories. This gives me a "fingerprint" of how much I tend to like songs from each genre, which helps the model make better predictions.

#### Key Features:
- **Handling Multi-Genre Tracks**: Tracks with multiple genres are split, encoded individually, and then aggregated.
- **Smoothing**: I smooth the encoding to prevent overfitting by balancing genre-specific means with the global mean.
- **Rare Genres**: Genres that appear fewer than 28 times are grouped into a common "Rare_Genre" category.
- **Unknown Genres**: Tracks with "Unknown" genres are handled separately, using the global mean or a fallback value for NMF rows.

In [ ]:
# Define explicitly disliked genres with severity weights (1-10 scale, 10 being most disliked)
disliked_genres = {
    'drone': 9, 'psychedelic': 7, 'improv': 8, 'ambient': 6, 'experimental': 7,
    'instrumental': 5, 'classical': 5, 'hardcore': 8, 'downtempo': 6, 'slowcore': 6,
    'noise': 9, 'satanic': 10, 'pagan': 8, 'metalcore': 9, 'deathcore': 10, 
    'death metal': 10, 'metal': 8, 'metallic hardcore': 9, 'beatdown deathcore': 10, 
    'nydm': 9, 'soundscape': 7, 'alternative metal': 8, 'horror punk': 8, 
    'sludge metal': 9, 'thrash metal': 9, 'death thrash metal': 10, 
    'heavy metal': 8, 'black metal': 10, 'doom metal': 9, 'death doom metal': 10,
    'techno': 8, 'hard techno': 9, 'tech house': 7, 'minimal techno': 8,
    'acid techno': 8, 'industrial techno': 9,  
    'psychedelic rock': 7, 'psychedelic pop': 7, 'neo-psychedelic': 7, 
    'psychedelic folk': 7, 'psychedelia': 7, 'psych': 7, 'psych rock': 7, 
    'psych pop': 7, 'psychedelic soul': 7, 'acid rock': 7
}

preferred_genres = {
    'alternative r&b': 10, 'chamber pop': 9, 'bedroom pop': 8.5,
    'indie folk': 8, 'post punk': 7, 'indie': 6.5, 'jangle pop': 6.3,
    'retro soul': 5, 'folk pop': 3.5, 'indie rock': 3.5, 'indie pop': 3.5,
    'baroque pop': 3.5, 'americana': 3
}

def target_encode_multi_genre(df, genre_column, target, smoothing=1, aggregation_method='mean', min_count=28):
    # Make a copy of the dataframe to avoid SettingWithCopyWarning
    df = df.copy()
    
    # Target encode a multi-genre column with improved handling of disliked genres
    df_train = df[df['playlist_origin'] != 'df_nmf'].copy()
    
    # Check if we have training data
    if len(df_train) == 0:
        raise ValueError("No training data found (df_train is empty)")
    
    global_mean = df_train[target].mean()
    
    # Split genres into lists
    df_train['split_genres'] = df_train[genre_column].fillna('Unknown').str.split(', ').apply(
        lambda x: [genre.strip() for genre in x if genre not in ['seen live', 'Unknown', '']] 
        if isinstance(x, list) else ['Unknown']
    )
    
    # Explode genres for encoding calculation
    exploded_genres = df_train.explode('split_genres')
    
    # Calculate genre statistics
    label_means = exploded_genres.groupby('split_genres')[target].mean()
    label_counts = exploded_genres['split_genres'].value_counts()
    rare_genres = label_counts[label_counts < min_count].index.tolist()
    
    # Apply smoothing
    smoothed_values = (label_means * label_counts + global_mean * smoothing) / (label_counts + smoothing)
    
    # Adjust rare genres
    for genre in rare_genres:
        if genre in smoothed_values:
            smoothed_values[genre] = (smoothed_values[genre] + global_mean * 2) / 3
    
    genre_encoding_map = smoothed_values.to_dict()
    
    def encode_genres(genres_str):
        if pd.isna(genres_str) or genres_str == 'Unknown':
            return global_mean
        
        genres = str(genres_str).split(', ')
        genres = [g.strip() for g in genres if g not in ['seen live', 'Unknown', '']]
        
        # Handle genres without commas
        if len(genres) == 1 and ' ' in genres[0]:
            genre_lower = genres[0].lower()
            # First check preferred genres
            for preferred_term in preferred_genres:
                if preferred_term in genre_lower:
                    return min(100, global_mean * 1.5)  # Boost preferred genres
            # Then check disliked terms
            for disliked_term in ['metal', 'satanic', 'gore', 'brutal']:
                if disliked_term in genre_lower:
                    return 30.0
        
        if not genres:
            return global_mean
        
        genre_values = []
        genre_weights = []
        
        for genre in genres:
            value = genre_encoding_map.get(genre, global_mean)
            weight = 1.0
            
            # Apply blessing for preferred genres
            genre_lower = genre.lower()
            if genre_lower in preferred_genres:
                blessing_factor = 1 + (preferred_genres[genre_lower] / 10.0)
                value = min(100, value * blessing_factor)  # Cap at 100
                weight = 1.5  # Higher weight for preferred genres
            elif genre_lower in disliked_genres:
                severity = disliked_genres.get(genre_lower, 6)
                penalty_factor = 1.0 - (severity / 10.0)
                value = value * penalty_factor
                weight = 1.0 + (severity / 10.0)
            elif genre in label_means and label_means[genre] < global_mean - 10:
                value = value * 0.75
                
            genre_values.append(value)
            genre_weights.append(weight)
        
        return sum(v * w for v, w in zip(genre_values, genre_weights)) / sum(genre_weights)
    
    # Apply encoding to ALL rows including NMF
    df[genre_column + '_encoded'] = df[genre_column].apply(encode_genres)
    return df

# Run the improved target encoding function 
try:
    df = target_encode_multi_genre(df, 'Genres', 'liked', smoothing=35, aggregation_method='mean')
except Exception as e:
    print(f"Error during encoding: {str(e)}")
    raise

# Check NMF rows encoding stats
if df is not None:
    nmf_encoding_check = df[df['playlist_origin'] == 'df_nmf'][['Track Name', 'Artist Name(s)', 'Album Name', 'Genres', 'Genres_encoded']]

    print(f"NMF rows total: {len(nmf_encoding_check)}")
    print(f"NMF rows with genre encoding: {nmf_encoding_check['Genres_encoded'].notna().sum()}")
    print(f"NMF rows missing genre encoding: {nmf_encoding_check['Genres_encoded'].isna().sum()}")

    if nmf_encoding_check['Genres_encoded'].notna().any():
        print("\nEncoding value stats:")
        print(f"Min: {nmf_encoding_check['Genres_encoded'].min():.2f}")
        print(f"Max: {nmf_encoding_check['Genres_encoded'].max():.2f}")
        print(f"Mean: {nmf_encoding_check['Genres_encoded'].mean():.2f}")
        print(f"Median: {nmf_encoding_check['Genres_encoded'].median():.2f}")
        
        diverse_samples = nmf_encoding_check.drop_duplicates(['Artist Name(s)'])[:10]
        print("\n10 tracks from different artists:")
        for i, row in diverse_samples.iterrows():
            print(f"{row['Artist Name(s)']} - {row['Track Name']} | Genres: {row['Genres']} | Score: {row['Genres_encoded']:.2f}")
        
        print("\nTop 5 highest genre scores:")
        high_scores = nmf_encoding_check.nlargest(5, 'Genres_encoded')
        for i, row in high_scores.iterrows():
            print(f"{row['Artist Name(s)']} - {row['Track Name']} | Genres: {row['Genres']} | Score: {row['Genres_encoded']:.2f}")
        
        print("\nBottom 5 lowest genre scores:")
        low_scores = nmf_encoding_check.nsmallest(5, 'Genres_encoded')
        for i, row in low_scores.iterrows():
            print(f"{row['Artist Name(s)']} - {row['Track Name']} | Genres: {row['Genres']} | Score: {row['Genres_encoded']:.2f}")
    else:
        print("\nNo encoded values found in NMF rows.")

    # Save results
    df[df['playlist_origin'] == 'df_nmf'][['Track Name', 'Artist Name(s)', 'Album Name', 'Genres', 'Genres_encoded']].to_csv('genre_encoding_results.csv', index=False)
    print("\nGenre encoding results saved to 'genre_encoding_results.csv'")
else:
    print("Dataframe is None - encoding failed")

# Finding How Central an Artist is to My Music Taste 🎯

To understand how central an artist is to my music taste, we're building a network of artists based on which ones I've liked and which are similar to them. Using **PageRank**, a method that measures the importance of nodes in a network, we calculate each artist's "centrality" score. This score reflects how influential an artist is within the network of my liked and similar artists.

## Key Points:
* **Liked Artists**: Artists from my liked songs and favorite albums.
* **Similar Artists**: Artists similar to my liked artists (from `df_liked_similar`).
* **No Data Leakage**: The `df_nmf` (New Music Friday) artists are excluded from the network to avoid bias.
* **Scaled Scores**: Centrality scores are normalized to a 0-100 range for easier interpretation.

This approach ensures that the centrality scores are based solely on my preferences and not influenced by artists I haven't liked or the New Music Friday playlist. Let's dive into the code! 🚀

In [ ]:
# Create the Featured_Artist(s) column
def extract_featured_artists(df):
    # Create new column by splitting on comma and taking all but first artist
    df['Featured_Artist(s)'] = df['Artist Name(s)'].apply(
        lambda x: ', '.join(str(x).split(',')[1:]).strip() if ',' in str(x) else ''
    )
    return df

# Prepare the data
def prepare_featured_artists(df):
    # Prepare the 'Featured_Artist(s)' column for analysis.
    # Ensures the column exists, handles missing values, and splits into lists. 🧹
    # First create the column if it doesn't exist
    df = extract_featured_artists(df)
    
    # Ensure 'Featured_Artist(s)' is a string and handle missing values
    df['Featured_Artist(s)'] = df['Featured_Artist(s)'].fillna('').astype(str)
    
    # Split and clean the 'Featured_Artist(s)' column into lists
    df['Featured_Artist(s)'] = df['Featured_Artist(s)'].str.split(',').apply(
        lambda x: [artist.strip() for artist in x] if isinstance(x, list) else []
    )
    return df

def build_graph(df, df_liked_similar, include_nmf=False):
    # Build a graph of artists and their connections.
    # Only includes liked artists and their similar artists by default. 🎯
    # Optionally includes NMF, not-liked, and mid artists (without adding edges). 🚫
    G = nx.Graph()
    
    # Add nodes for liked artists
    liked_artists = set(
        df[df['playlist_origin'].isin(['df_liked', 'df_fav_albums'])]['Artist Name(s)']
        .str.split(',').explode().str.strip()
    )
    G.add_nodes_from(liked_artists, type='liked')
    
    # Add nodes for similar artists (from liked)
    similar_artists_liked = set(
        df_liked_similar['Similar Artists']
        .dropna()
        .str.split(',').explode().str.strip()
    )
    G.add_nodes_from(similar_artists_liked, type='similar_liked')
    
    # Add edges based on similarity (from liked)
    for _, row in df_liked_similar.iterrows():
        artist = row['Artist']
        if isinstance(row['Similar Artists'], str):
            similar = row['Similar Artists'].split(', ')
            for s in similar:
                G.add_edge(artist, s, weight=1.0)
    
    # Optionally include NMF, not-liked, and mid artists (without adding edges)
    if include_nmf:
        nmf_artists = set(
            df[df['playlist_origin'] == 'df_nmf']['Artist Name(s)']
            .str.split(',').explode().str.strip()
        )
        not_liked_artists = set(
            df[df['playlist_origin'] == 'df_not_liked']['Artist Name(s)']
            .str.split(',').explode().str.strip()
        )
        mid_artists = set(
            df[df['playlist_origin'] == 'df_mid']['Artist Name(s)']
            .str.split(',').explode().str.strip()
        )
        G.add_nodes_from(nmf_artists, type='nmf')
        G.add_nodes_from(not_liked_artists, type='not_liked')
        G.add_nodes_from(mid_artists, type='mid')  # Mid artists exist but don’t interfere with the web
    
    return G

def calculate_centrality_scores(G, df):
    # Calculate PageRank centrality for all artists in the graph.
    # Maps centrality scores back to the DataFrame for main artists. 📊
    centrality_scores = nx.pagerank(G)
    
    # Map centrality scores back to DataFrame for main artists
    df['Artist Centrality'] = (
        df['Artist Name(s)']
        .str.split(',').str[0].str.strip()
        .map(centrality_scores).fillna(0)
    )
    
    return df, centrality_scores

# Normalize centrality scores to a 0-100 range
def normalize_centrality_scores(df):
    if df['Artist Centrality'].max() != 0:
        df['Artist Centrality'] = (df['Artist Centrality'] / df['Artist Centrality'].max()) * 100
    return df

# Run the complete pipeline
def run_centrality_analysis(df, df_liked_similar, include_nmf=False):
    # Run the complete centrality analysis pipeline:
    # 1. Prepare featured artists data.
    # 2. Build the graph.
    # 3. Calculate centrality scores.
    # 4. Normalize scores to 0-100 range. 🚀
    # Prepare the featured artists data (still needed to ensure the column exists)
    df = prepare_featured_artists(df)
    
    # Build the graph and calculate centrality
    G = build_graph(df, df_liked_similar, include_nmf=include_nmf)
    df, centrality_scores = calculate_centrality_scores(G, df)
    df = normalize_centrality_scores(df)
    
    return df, G, centrality_scores

# Execute the analysis for all artists (including NMF, not-liked, and mid)
df, G, centrality_scores = run_centrality_analysis(df, df_liked_similar, include_nmf=True)

# Extract the primary artist (first artist in 'Artist Name(s)' field)
df['Primary Artist'] = df['Artist Name(s)'].str.split(',').str[0].str.strip()

# Now group by Primary Artist to ensure unique artists are considered
df_unique_artists = df.groupby('Primary Artist', as_index=False)['Artist Centrality'].max()

# Get the top 30 unique primary artists by their centrality score
top_artist_df = df_unique_artists.nlargest(30, 'Artist Centrality')[['Primary Artist', 'Artist Centrality']]

# Print the top 30 unique primary artists by centrality
print("\nTop 30 Unique Primary Artists by Centrality:")
top_artist_df

### Distribution of Artist Centrality Scores by Playlist Origin 📊🎵

This histogram shows how **Artist Centrality** scores are distributed across different playlists. The centrality score reflects how central an artist is to my music taste, based on their connections to my liked artists and their similar artists.

- **Liked Artists (df_liked)**: Artists I’ve explicitly liked on Spotify.
- **Favorite Albums (df_fav_albums)**: Albums I’ve enjoyed in recent years.
- **Mid Albums (df_mid)**: Albums I felt were just okay. Not gbad, not great either.
- **Not Liked Albums (df_not_liked)**: Albums I didn’t enjoy in recent years.
- **New Music Friday (df_nmf)**: The most recent New Music Friday playlist.

#### What We’re Looking For:
- **High Centrality Scores**: Artists who are closely connected to my liked artists.
- **Low Centrality Scores**: Artists who are less connected to my preferences.
- **Patterns by Playlist**: Do certain playlists (e.g., liked vs. not liked) have distinct centrality distributions?

This visualization helps us understand how my preferences are reflected in the network of artists and how new or not-liked artists compare to my favorites. 🕵️‍♂️✨

In [ ]:
# Set figure size
plt.figure(figsize=(10, 5))

# Plot the distribution of 'Artist Centrality', colored by playlist_origin
# This shows how centrality scores vary across different playlists 🎨
sns.histplot(df, x='Artist Centrality', bins=30, hue='playlist_origin', kde=True, alpha=0.6, palette='viridis')

# Labels and title
plt.xlabel('Artist Centrality Score')
plt.ylabel('Count')
plt.title('Distribution of Artist Centrality Scores by Playlist Origin')

# Show the plot
plt.show()

### Lucy Dacus is one of my most played artists of all time, let's look at what her Pagerank network looked like!

In [ ]:
# Extract Lucy Dacus's centrality score
lucy_dacus_centrality = df[df['Artist Name(s)'].str.contains('Lucy Dacus')]['Artist Centrality'].values[0]

# Extract Lucy Dacus's connections from the graph
lucy_dacus_connections = list(G.edges('Lucy Dacus'))

# Print Lucy Dacus's centrality score and connections
print(f"Lucy Dacus's Centrality Score: {lucy_dacus_centrality:.2f}")
print(f"Lucy Dacus's Connections: {lucy_dacus_connections}")

# Create a subgraph centered around Lucy Dacus
lucy_dacus_subgraph = G.subgraph(['Lucy Dacus'] + [artist for edge in lucy_dacus_connections for artist in edge])

# Set up the plot
plt.figure(figsize=(12, 8))

# Draw the network
pos = nx.spring_layout(lucy_dacus_subgraph, seed=42)  # Layout for consistent positioning
nx.draw_networkx_nodes(lucy_dacus_subgraph, pos, node_size=500, node_color='lightblue', alpha=0.9)
nx.draw_networkx_edges(lucy_dacus_subgraph, pos, edge_color='gray', alpha=0.6)

# Highlight Lucy Dacus
nx.draw_networkx_nodes(lucy_dacus_subgraph, pos, nodelist=['Lucy Dacus'], node_size=1000, node_color='purple', alpha=0.9)
nx.draw_networkx_labels(lucy_dacus_subgraph, pos, labels={'Lucy Dacus': 'Lucy Dacus'}, font_size=12, font_weight='bold')

# Add labels for connected artists
nx.draw_networkx_labels(lucy_dacus_subgraph, pos, font_size=10)

# Add title and annotations
plt.title("Lucy Dacus's Position in My Music Taste Network", fontsize=16)
plt.figtext(0.5, 0.05, f"Centrality Score: {lucy_dacus_centrality:.2f}", ha='center', fontsize=12, style='italic')

# Remove axes
plt.axis('off')

# Show the plot
plt.show()

## Two More Features Before Primetime! 🎭

**Mood Score**: Combines Valence, Danceability, and Liveness to capture the vibe.

**Energy Profile**: Mashes Energy, Loudness, and Tempo to gauge the track’s intensity.


In [ ]:
# Calculate features only on non-NMF data
non_nmf_df = df[df['playlist_origin'] != 'df_nmf'].copy()

# Create mood_score and energy_profile on non-NMF data
non_nmf_df['mood_score'] = non_nmf_df[['Valence', 'Danceability', 'Liveness']].mean(axis=1)
non_nmf_df['energy_profile'] = non_nmf_df[['Energy', 'Loudness', 'Tempo']].mean(axis=1)

# Merge these features back into the main dataframe
df = df.merge(non_nmf_df[['mood_score', 'energy_profile']], how='left', left_index=True, right_index=True)

## A Clean Album ID

In [ ]:
# Create an Album ID for better data tracking for New Music Friday albums
df["Primary Artist"] = df["Artist Name(s)"].str.split(",").str[0].str.strip()

# ADD THIS DIRECTLY AFTER (same indentation):
df['album_id'] = df.apply(
    lambda x: f"{x['Primary Artist'].lower()}_{x['Album Name'].lower()}".replace(' ', '_'),
    axis=1
)

## Previwing the features on the menu 📖

In [ ]:
print("Columns in the DataFrame:")
print(df.columns)

print("\nData Types of Each Column:")
print(df.dtypes)

## Standardize the numeric columns 📏
When some numbers have a larger size than others, the model can be biased towards them, so we bring all the numeric columns on a similar scale.

### Seperate New Music Friday and Save it for Later!

In [ ]:
# Calculate mood and energy scores first
df['mood_score'] = df[['Valence', 'Danceability', 'Liveness']].mean(axis=1)
df['energy_profile'] = df[['Energy', 'Loudness', 'Tempo']].mean(axis=1)

# Split and save data
df_nmf = df[df['playlist_origin'] == 'df_nmf'].copy()
df = df[df['playlist_origin'] != 'df_nmf'].copy()

# Save both versions pre-standardization
df_nmf.to_csv('data/df_nmf_later.csv', index=False)
df_cleaned_pre_standardized = pd.concat([df, df_nmf], ignore_index=True)
df_cleaned_pre_standardized.to_csv('data/df_cleaned_pre_standardized.csv', index=False)

# Store original values
original_centrality = df_nmf['Artist Centrality'].copy()
original_mood = df_nmf['mood_score'].copy()
original_energy = df_nmf['energy_profile'].copy()

In [ ]:
# Define numeric columns to scale
numeric_columns = [ 
    'Genres_encoded', 
    'Artist Centrality',
    'Popularity',
    'Record Label Frequency Encoded', 
    'mood_score', 
    'energy_profile'
]

# Initialize the scaler with a 1 to 100 range
scaler = MinMaxScaler(feature_range=(1, 100))

# Fit the scaler on the training data (df)
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# Transform the test data (df_nmf) using the fitted scaler
df_nmf[numeric_columns] = scaler.transform(df_nmf[numeric_columns])

# Save the scaled df_nmf for later use
df_nmf.to_csv('data/df_nmf_later.csv', index=False)

# Display the first few rows of the scaled test data
print("\nScaled Test Data (df_nmf):")
df_nmf[numeric_columns].head()

## Tuning and Predicting with Random Forest & XGBoost 🌟
In this section, we fine-tune our Random Forest and XGBoost models using randomized search for optimal hyperparameters. The goal? To get the best possible performance in predicting song ratings. After tuning the models, we make predictions on the unseen data, combining both models' results to generate a more accurate score!

In [ ]:
def tune_models(df, features, test_size=0.2):
    # Prepare data
    X = df[features]
    y = (df['liked'] - df['liked'].mean()) / df['liked'].std()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    # Model parameters
    rf_params = {
        'n_estimators': randint(50, 300),
        'max_depth': randint(3, 15),
        'min_samples_split': randint(2, 10),
        'min_samples_leaf': randint(1, 5)
    }
    
    xgb_params = {
        'n_estimators': randint(50, 300),
        'max_depth': randint(3, 10),
        'learning_rate': uniform(0.01, 0.3),
        'subsample': uniform(0.6, 0.4),
        'colsample_bytree': uniform(0.6, 0.4)
    }
    
    # Initialize models
    rf = RandomForestRegressor(random_state=42)
    xgb = XGBRegressor(random_state=42)
    
    # Perform randomized search
    rf_search = RandomizedSearchCV(rf, rf_params, n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)
    xgb_search = RandomizedSearchCV(xgb, xgb_params, n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)
    
    # Fit models
    rf_search.fit(X_train, y_train)
    xgb_search.fit(X_train, y_train)
    
    # Train final models with best parameters
    best_rf = RandomForestRegressor(**rf_search.best_params_, random_state=42)
    best_xgb = XGBRegressor(**xgb_search.best_params_, random_state=42)
    
    best_rf.fit(X_train, y_train)
    best_xgb.fit(X_train, y_train)
    
    return best_rf, best_xgb, rf_search.best_params_, xgb_search.best_params_, X_test, y_test

def predict_with_tuned_models(best_rf, best_xgb, df_nmf, features, y_mean, y_std):
    # Make predictions
    rf_pred = best_rf.predict(df_nmf[features]) * y_std + y_mean
    xgb_pred = best_xgb.predict(df_nmf[features]) * y_std + y_mean
    
    # Combine predictions
    df_nmf['predicted_score'] = (rf_pred + xgb_pred) / 2
    
    return df_nmf

# Usage
features = [
    'Genres_encoded', 
    'Artist Centrality',  
    'Record Label Frequency Encoded', 
    'mood_score', 
    'Popularity',
    'energy_profile'
]
best_rf, best_xgb, rf_params, xgb_params, X_test, y_test = tune_models(df, features)
df_nmf = predict_with_tuned_models(best_rf, best_xgb, df_nmf, features, df['liked'].mean(), df['liked'].std())

# Output best parameters and feature importances
print(f"Random Forest best parameters: {rf_params}")
print(f"XGBoost best parameters: {xgb_params}")
print(f"Random Forest feature importances: {best_rf.feature_importances_}")
print(f"XGBoost feature importances: {best_xgb.feature_importances_}")

# Feature importance DataFrame
feature_importance = pd.DataFrame({'feature': features, 'importance': best_rf.feature_importances_}).sort_values('importance', ascending=False)
print("Feature importance (Random Forest):")
print(feature_importance)

# 80/20 Train/Test of The Non NMF Data using RandomForrest and XGBoost

In [ ]:
def train_test_model(df, features, rf_params, xgb_params):
    # Only use non-NMF data for training
    train_df = df[df['playlist_origin'] != 'df_nmf'].copy()
    
    # Prepare features and target
    X = train_df[features]
    y = train_df['liked']
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Initialize models with the best parameters from tuning
    rf_model = RandomForestRegressor(**rf_params, random_state=42)
    xgb_model = XGBRegressor(**xgb_params, random_state=42)
    
    # Train models
    rf_model.fit(X_train, y_train)
    xgb_model.fit(X_train, y_train)
    
    # Make predictions
    rf_pred = rf_model.predict(X_test)
    xgb_pred = xgb_model.predict(X_test)
    
    # Combine predictions (80/20 weight)
    final_pred = (0.8 * rf_pred) + (0.2 * xgb_pred)
    
    # Calculate metrics
    mse = mean_squared_error(y_test, final_pred)
    r2 = r2_score(y_test, final_pred)
    
    # Create results DataFrame
    results_df = pd.DataFrame({
        'Track Name': train_df.loc[X_test.index, 'Track Name'],
        'Artist Name(s)': train_df.loc[X_test.index, 'Artist Name(s)'],
        'Actual Score': y_test,
        'Predicted Score': final_pred
    })
    
    # Get feature importances
    importances = pd.DataFrame({
        'Feature': features,
        'RF Importance': rf_model.feature_importances_,
        'XGB Importance': xgb_model.feature_importances_
    })
    
    # Print metrics
    print(f"\nModel Performance:")
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"R² Score: {r2:.2f}")
    
    print("\nTop 10 Predictions vs Actual:")
    print(results_df.sort_values('Predicted Score', ascending=False).head(10))
    
    print("\nFeature Importances:")
    print(importances.sort_values('RF Importance', ascending=False))
    
    return rf_model, xgb_model, results_df, importances

# Usage
features = ['Genres_encoded', 'Artist Centrality', 'Record Label Frequency Encoded', 'mood_score', 'Popularity', 'energy_profile']
rf_model, xgb_model, results, importances = train_test_model(df, features, rf_params, xgb_params)

## Run the New Music Friday Regression Model

We're using non-NMF data to train Random Forest and XGBoost models to predict how much users will like different tracks. Our improved model includes two key enhancements:

1. **Artist Similarity Boost**: We reward New Music Friday artists that are similar to your liked artists, helping you discover new music that aligns with your established preferences.

2. **Adaptive Ensemble Weighting**: Instead of a fixed 50/50 blend between models, we dynamically adjust weights based on which model performs better for different types of music.

After training, we make predictions for new tracks and calculate confidence intervals to gauge prediction reliability. We then aggregate results by album, factoring in consistency and track count, and sort by weighted score to create a personalized list of top album recommendations.


In [ ]:
def calculate_adaptive_weights(X_train, y_train, X_pred, rf_model, xgb_model, features, n_bins=5):
    # Make predictions on training data
    rf_train_pred = rf_model.predict(X_train)
    xgb_train_pred = xgb_model.predict(X_train)
    
    # Calculate errors
    rf_errors = np.abs(rf_train_pred - y_train)
    xgb_errors = np.abs(xgb_train_pred - y_train)
    
    # Create a dataframe with features, predictions, and errors
    error_df = X_train.copy()
    error_df['rf_error'] = rf_errors
    error_df['xgb_error'] = xgb_errors
    
    # Get top 2 features by importance
    feature_importance = pd.DataFrame({
        'feature': features,
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    top_features = feature_importance['feature'].head(2).tolist()
    
    # Create bins for each top feature
    for feature in top_features:
        error_df[f'{feature}_bin'] = pd.qcut(error_df[feature], n_bins, labels=False, duplicates='drop')
    
    # Calculate performance by bin
    performance_by_bin = {}
    for feature in top_features:
        performance_by_bin[feature] = {}
        for bin_idx in range(n_bins):
            bin_data = error_df[error_df[f'{feature}_bin'] == bin_idx]
            if len(bin_data) > 0:
                rf_mean_error = bin_data['rf_error'].mean()
                xgb_mean_error = bin_data['xgb_error'].mean()
                total_error = rf_mean_error + xgb_mean_error
                
                if total_error > 0:
                    # Weight inversely proportional to error (higher weight = better model)
                    rf_weight = xgb_mean_error / total_error
                else:
                    rf_weight = 0.5
                
                performance_by_bin[feature][bin_idx] = rf_weight
    
    # Calculate weights for prediction data
    weights = np.ones(len(X_pred)) * 0.5  # Default 50/50 weight
    
    for feature in top_features:
        # Create bins for prediction data
        try:
            pred_bins = pd.qcut(X_pred[feature], n_bins, labels=False, duplicates='drop')
            
            # Apply weights based on bin performance
            for bin_idx in range(n_bins):
                if bin_idx in performance_by_bin[feature]:
                    bin_mask = (pred_bins == bin_idx)
                    if any(bin_mask):
                        rf_weight = performance_by_bin[feature][bin_idx]
                        # Scale weight to 0.3-0.7 range to avoid extreme values
                        scaled_weight = 0.3 + (rf_weight * 0.4)
                        weights[bin_mask] = scaled_weight
        except Exception as e:
            print(f"Warning: Could not calculate bins for {feature}: {e}")
            continue
    
    return weights

def calculate_artist_similarity_boost(df_nmf, df_liked_similar, liked_artists_df, boost_factor=15):
    # Create a copy to avoid modifying the original
    df_nmf_copy = df_nmf.copy()
    
    # Extract primary artists from NMF
    df_nmf_copy['Primary Artist'] = df_nmf_copy['Artist Name(s)'].str.split(',').str[0].str.strip()
    
    # Extract primary artists from liked songs and albums
    liked_artists = set(liked_artists_df['Artist Name(s)'].str.split(',').str[0].str.strip())
    
    # Create a dictionary of similar artists from df_liked_similar
    similar_artists_dict = {}
    for _, row in df_liked_similar.iterrows():
        if pd.notna(row['Similar Artists']):
            artist = row['Artist']
            similar = [s.strip() for s in row['Similar Artists'].split(',')]
            similar_artists_dict[artist] = similar
    
    # Flatten the similar artists dictionary to create a mapping from similar artist to liked artist
    similar_to_liked_mapping = {}
    for liked_artist, similar_list in similar_artists_dict.items():
        for similar_artist in similar_list:
            if similar_artist not in similar_to_liked_mapping:
                similar_to_liked_mapping[similar_artist] = []
            similar_to_liked_mapping[similar_artist].append(liked_artist)
    
    # Function to find the best match for an artist using fuzzy matching
    def find_best_match(artist, artist_set, threshold=85):
        if artist in artist_set:
            return artist, 100
        
        matches = process.extractOne(artist, artist_set)
        if matches and matches[1] >= threshold:
            return matches[0], matches[1]
        return None, 0
    
    # Add similarity boost columns directly to the dataframe
    df_nmf_copy['similarity_boost'] = 0
    df_nmf_copy['match_type'] = 'no_match'
    df_nmf_copy['matched_to'] = 'None'
    
    # Calculate similarity boost for each NMF artist
    for i, row in df_nmf_copy.iterrows():
        artist = row['Primary Artist']
        
        # Check if the artist is directly in liked artists
        if artist in liked_artists:
            df_nmf_copy.at[i, 'similarity_boost'] = boost_factor
            df_nmf_copy.at[i, 'match_type'] = 'direct_match'
            df_nmf_copy.at[i, 'matched_to'] = artist
            continue
        
        # Check if the artist is in similar_to_liked_mapping
        if artist in similar_to_liked_mapping:
            liked_connections = similar_to_liked_mapping[artist]
            df_nmf_copy.at[i, 'similarity_boost'] = boost_factor * 0.8
            df_nmf_copy.at[i, 'match_type'] = 'similar_to_liked'
            df_nmf_copy.at[i, 'matched_to'] = ', '.join(liked_connections[:3])  # Show up to 3 connections
            continue
        
        # Try fuzzy matching with similar artists
        best_match, match_score = find_best_match(artist, set(similar_to_liked_mapping.keys()))
        if best_match:
            liked_connections = similar_to_liked_mapping[best_match]
            df_nmf_copy.at[i, 'similarity_boost'] = boost_factor * 0.6 * (match_score / 100)
            df_nmf_copy.at[i, 'match_type'] = 'fuzzy_similar_match'
            df_nmf_copy.at[i, 'matched_to'] = f"{best_match} ({match_score}%) -> {', '.join(liked_connections[:2])}"
            continue
        
        # Try fuzzy matching with liked artists
        best_match, match_score = find_best_match(artist, liked_artists)
        if best_match:
            df_nmf_copy.at[i, 'similarity_boost'] = boost_factor * 0.7 * (match_score / 100)
            df_nmf_copy.at[i, 'match_type'] = 'fuzzy_direct_match'
            df_nmf_copy.at[i, 'matched_to'] = f"{best_match} ({match_score}%)"
            continue
    
    return df_nmf_copy

def get_prediction_interval(X, model, y_std, y_mean, percentile=95):
    """Calculate prediction intervals from Random Forest"""
    predictions = []
    for estimator in model.estimators_:
        predictions.append(estimator.predict(X) * y_std + y_mean)
    predictions = np.array(predictions)
    lower = np.percentile(predictions, (100-percentile)/2, axis=0)
    upper = np.percentile(predictions, 100-(100-percentile)/2, axis=0)
    return lower, upper

def apply_confidence_scaling(predictions, model, X, y_std, y_mean, min_confidence=40):
    """Apply confidence-based scaling to predictions"""
    # Get prediction intervals
    lower, upper = get_prediction_interval(X, model, y_std, y_mean)
    uncertainty = upper - lower
    confidence = (1 - uncertainty/uncertainty.max()) * 100
    
    # Scale predictions based on confidence
    scaled_predictions = predictions.copy()
    
    # High confidence: minimal adjustment (90-100% of original)
    high_conf = confidence >= min_confidence
    scaled_predictions[high_conf] = predictions[high_conf] * (0.9 + 0.1*(confidence[high_conf]/100))
    
    # Low confidence: aggressive scaling (60-90% of original)
    low_conf = confidence < min_confidence
    scaled_predictions[low_conf] = predictions[low_conf] * (0.6 + 0.3*(confidence[low_conf]/min_confidence))
    
    return np.clip(scaled_predictions, None, 100), confidence

# Load the similar artists data
df_liked_similar = pd.read_csv('data/liked_artists_only_similar.csv')

# Filter liked and favorite albums for similarity boost
liked_artists_df = df[df['playlist_origin'].isin(['df_liked', 'df_fav_albums'])]

# Create known artists set
known_artists = set(df_liked_similar['Artist'])

# Features used for prediction
features = [
    'Genres_encoded', 
    'Artist Centrality',  
    'Record Label Frequency Encoded', 
    'mood_score', 
    'Popularity',
    'energy_profile'
]

# Normalize the target variable
y_mean = df['liked'].mean()
y_std = df['liked'].std()
y_normalized = (df['liked'] - y_mean) / y_std

# Prepare training data
X = df[features]
y = y_normalized  # Use normalized target

# Initialize models with the best parameters from tuning
rf_model = RandomForestRegressor(**rf_params, random_state=42)
xgb_model = XGBRegressor(**xgb_params, random_state=42)

# Train models
rf_model.fit(X, y)
xgb_model.fit(X, y)

# Get feature importance from both models
rf_importance = pd.DataFrame({
    'feature': features,
    'importance_rf': rf_model.feature_importances_
}).sort_values('importance_rf', ascending=False)

xgb_importance = pd.DataFrame({
    'feature': features,
    'importance_xgb': xgb_model.feature_importances_
}).sort_values('importance_xgb', ascending=False)

# Combine importance scores
feature_importance = pd.merge(rf_importance, xgb_importance, on='feature')
feature_importance['avg_importance'] = (feature_importance['importance_rf'] + feature_importance['importance_xgb']) / 2
feature_importance = feature_importance.sort_values('avg_importance', ascending=False)

# Prepare NMF data for prediction
df_nmf_cleaned = df_nmf[features]

# Calculate similarity boost for NMF artists
df_nmf = calculate_artist_similarity_boost(df_nmf, df_liked_similar, liked_artists_df)

# Make predictions and denormalize
rf_predictions = rf_model.predict(df_nmf_cleaned) * y_std + y_mean
xgb_predictions = xgb_model.predict(df_nmf_cleaned) * y_std + y_mean

# Calculate adaptive weights for ensemble
adaptive_weights = calculate_adaptive_weights(X, y_normalized, df_nmf_cleaned, rf_model, xgb_model, features)

# Get raw ensemble predictions
raw_predictions = (adaptive_weights * rf_predictions + 
                  (1 - adaptive_weights) * xgb_predictions)

# Apply similarity boost
boosted_predictions = raw_predictions + df_nmf['similarity_boost']

# Apply confidence scaling
final_predictions, confidence_scores = apply_confidence_scaling(
    boosted_predictions, 
    rf_model, 
    df_nmf_cleaned, 
    y_std, 
    y_mean,
    min_confidence=40
)

# Store results
df_nmf['predicted_score'] = final_predictions
df_nmf['prediction_confidence'] = confidence_scores
df_nmf['is_unknown_artist'] = ~df_nmf['Artist Name(s)'].isin(known_artists)

# Calculate prediction intervals
lower_bound, upper_bound = get_prediction_interval(df_nmf_cleaned, rf_model, y_std, y_mean)
df_nmf['prediction_lower'] = lower_bound
df_nmf['prediction_upper'] = upper_bound
df_nmf['prediction_uncertainty'] = upper_bound - lower_bound

# Get the most common release date from NMF dataset
nmf_release_date = df_nmf['Release Date'].mode().iloc[0]

# Function to find common artists across all tracks in an album
def get_common_artists(artist_series):
    """Find artists common to all tracks in an album"""
    # Handle NaN/None values
    artist_series = artist_series.dropna()
    if len(artist_series) == 0:
        return ""
    
    # Split each string into a set of artists
    artist_lists = []
    for artists in artist_series:
        try:
            artist_lists.append(set(a.strip() for a in artists.split(',')))
        except AttributeError:
            continue
    
    if not artist_lists:
        return ""
    
    # Find intersection of all artist sets
    common_artists = set.intersection(*artist_lists)
    
    # If no common artists, use most frequent artist
    if not common_artists:
        all_artists = [a for artist_set in artist_lists for a in artist_set]
        artist_counts = Counter(all_artists)
        
        # Get artists that appear in all tracks
        most_common = [a for a, cnt in artist_counts.items() 
                      if cnt == len(artist_series)]
        
        if most_common:
            common_artists = set(most_common)
        else:
            # Fall back to single most common artist
            common_artists = {artist_counts.most_common(1)[0][0]}
    
    return ', '.join(sorted(common_artists))
    return ""

# Group by album and aggregate data
album_predictions = df_nmf.groupby('Album Name').agg({
    'Artist Name(s)': get_common_artists,
    'predicted_score': ['mean', 'std', 'count'],
    'prediction_uncertainty': 'mean',
    'Genres': lambda x: ' | '.join(list(set(x))[:3]),
    'Record Label': 'first',
    'Artist Centrality': 'first', 
    'mood_score': 'first',         
    'energy_profile': 'first',
    'Genres_encoded': 'first',
    'Record Label Frequency Encoded': 'first',
    'prediction_confidence': 'mean'
}).reset_index()

# Flatten column names
album_predictions.columns = [
    'Album Name', 'Artist', 'avg_score', 'score_std', 'track_count',
    'avg_uncertainty', 'Genres', 'Label', 'Artist_Centrality', 
    'Mood_Score', 'Energy_Profile', 'Genres_encoded', 
    'Record_Label_Frequency_Encoded', 'confidence_score'
]

# Calculate final confidence score
max_std = album_predictions['score_std'].max()
max_uncertainty = album_predictions['avg_uncertainty'].max()

album_predictions['confidence_score'] = (
    (1 - album_predictions['score_std'] / max_std) * 
    (1 - album_predictions['avg_uncertainty'] / max_uncertainty) * 
    (1 - 1/(1 + album_predictions['track_count']))
) * 100

album_predictions['confidence_score'] = np.clip(album_predictions['confidence_score'], 1, 100)

# Filter and sort albums
album_recommendations = album_predictions[album_predictions['track_count'] >= 5].sort_values('avg_score', ascending=False)

# Output results
output_columns = [
    'Artist', 'Album Name', 'avg_score', 'confidence_score',
    'track_count', 'Genres', 'Label', 'Artist_Centrality', 
    'Mood_Score', 'Energy_Profile'
]

final_recommendations = album_recommendations[output_columns].copy()
final_recommendations['Artist_Centrality'] = final_recommendations['Artist_Centrality'].clip(1, 100)
final_recommendations['Mood_Score'] = final_recommendations['Mood_Score'].clip(1, 100)
final_recommendations['Energy_Profile'] = final_recommendations['Energy_Profile'].clip(1, 100)

# Save recommendations
date_str = datetime.strptime(nmf_release_date, '%Y-%m-%d').strftime('%m-%d-%y')
filename = f"predictions/{date_str}_Album_Recommendations.csv"
final_recommendations.round(2).to_csv(filename, index=False)

# Evaluation metrics
def custom_scorer(y_true, y_pred):
    y_true_denormalized = y_true * y_std + y_mean
    y_pred_denormalized = y_pred * y_std + y_mean
    return -mean_squared_error(y_true_denormalized, y_pred_denormalized)

custom_scorer_func = make_scorer(custom_scorer, greater_is_better=False)
rf_cv_scores = cross_val_score(rf_model, X, y, cv=5, scoring=custom_scorer_func)
xgb_cv_scores = cross_val_score(xgb_model, X, y, cv=5, scoring=custom_scorer_func)

# Print results
print(f"\n=== New Music Friday Recommendations ({nmf_release_date}) ===")
print(f"Albums with 5+ tracks: {len(album_recommendations)}")
print("\nTop 20 Albums:")
print(album_recommendations[['Artist', 'Album Name', 'avg_score', 'track_count']].head(20).round(2).to_string(index=False))

print("\n=== Model Performance ===")
print(f"Random Forest CV Score: {rf_cv_scores.mean():.3f} (+/- {rf_cv_scores.std() * 2:.3f})")
print(f"XGBoost CV Score: {xgb_cv_scores.mean():.3f} (+/- {xgb_cv_scores.std() * 2:.3f})")

print("\n=== Feature Importance ===")
print(feature_importance[['feature', 'avg_importance']].round(3).to_string(index=False))

# Visualization
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['avg_importance'])
plt.xlabel('Average Importance')
plt.title('Top Features')
plt.gca().invert_yaxis()
plt.show()

# Display the Top Recommended Albums!

In [ ]:
# Folder where the predictions are saved
predictions_folder = "predictions/"

# Get the latest predictions file
files = [f for f in os.listdir(predictions_folder) if f.endswith("_Album_Recommendations.csv")]

if files:
    thisweek = max(files, key=lambda f: os.path.getmtime(os.path.join(predictions_folder, f)))
    print(f"Loaded latest file: {thisweek}")  # Optional, just for confirmation
    model_output_df = pd.read_csv(os.path.join(predictions_folder, thisweek))
    model_output_df = model_output_df.sort_values(by="avg_score", ascending=False)

    # Display all rows
    pd.set_option("display.max_rows", None)
    model_output_df
else:
    thisweek = None
    None  # Ensures no unwanted output

thisweek  # Stores the filename for reference
model_output_df  #Show the Recommendations Dataframe for This Past Week!

## Grab Album Art for the NMF Albums

In [ ]:
# My Last.fm API key
LASTFM_API_KEY = "74a510ecc9fc62bf3e0edc6adc2e99f9"

# Function to get similar artists using Last.fm API
def get_similar_artists(artist: str, api_key: str, limit: int = 5) -> dict:
    base_url = "http://ws.audioscrobbler.com/2.0/"
    params = {
        'method': 'artist.getsimilar',
        'artist': artist,
        'api_key': api_key,
        'limit': limit,
        'format': 'json'
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        
        if 'similarartists' in data and 'artist' in data['similarartists']:
            similar_artists = [artist['name'] for artist in data['similarartists']['artist']]
            return {'Artist': artist, 'Similar Artists': ", ".join(similar_artists[:limit]), 'status': 'success'}
    except Exception as e:
        return {'Artist': artist, 'Similar Artists': None, 'status': f'error: {str(e)}'}
    
    return {'Artist': artist, 'Similar Artists': None, 'status': 'no_results'}

# Function to get album art using Apple Music API
def get_album_art(artist: str, album: str) -> dict:
    try:
        url = f"https://itunes.apple.com/search?term={artist} {album}&entity=album"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        if 'results' in data and len(data['results']) > 0:
            album_art = data['results'][0].get('artworkUrl100', '').replace("100x100", "600x600")
            return {'Artist': artist, 'Album Name': album, 'Album Art': album_art, 'status': 'success'}
    except Exception as e:
        return {'Artist': artist, 'Album Name': album, 'Album Art': None, 'status': f'error: {str(e)}'}
    
    return {'Artist': artist, 'Album Name': album, 'Album Art': None, 'status': 'no_results'}

# Main function to update album data
def update_album_data(input_file: str, album_art_file: str, similar_artists_file: str, api_key: str) -> None:
    print(f"\nStarting data fetch at {datetime.now().strftime('%H:%M:%S')}")
    df_input = pd.read_csv(input_file)
    album_pairs = df_input[['Primary Artist', 'Album Name']].drop_duplicates()
    recommended_artists = df_input[df_input['playlist_origin'] == 'df_nmf']['Primary Artist'].unique()
    
    try:
        existing_album_art = pd.read_csv(album_art_file)
    except FileNotFoundError:
        existing_album_art = pd.DataFrame(columns=['Artist', 'Album Name', 'Album Art'])
    
    album_pairs = album_pairs.merge(
        existing_album_art,
        left_on=['Primary Artist', 'Album Name'],
        right_on=['Artist', 'Album Name'],
        how='left',
        indicator=True
    )
    album_pairs = album_pairs[album_pairs['_merge'] == 'left_only'].drop(columns=['_merge', 'Album Art'])
    
    df_album_art = pd.DataFrame(columns=['Artist', 'Album Name', 'Album Art'])
    df_similar = pd.DataFrame(columns=['Artist', 'Similar Artists'])
    
    with ThreadPoolExecutor(max_workers=4) as executor:
        future_to_album = {
            executor.submit(get_album_art, row['Primary Artist'], row['Album Name']): (row['Primary Artist'], row['Album Name'])
            for _, row in album_pairs.iterrows()
        }
        for future in as_completed(future_to_album):
            result = future.result()
            if result['status'] == 'success' and result['Album Art']:
                df_album_art = pd.concat([df_album_art, pd.DataFrame([result])], ignore_index=True)
            sleep(0.25)
    
    updated_album_art = pd.concat([existing_album_art, df_album_art], ignore_index=True)
    updated_album_art.to_csv(album_art_file, index=False)
    
    with ThreadPoolExecutor(max_workers=4) as executor:
        future_to_artist = {
            executor.submit(get_similar_artists, artist, api_key): artist
            for artist in recommended_artists
        }
        for future in as_completed(future_to_artist):
            result = future.result()
            if result['status'] == 'success' and result['Similar Artists']:
                df_similar = pd.concat([df_similar, pd.DataFrame([result])], ignore_index=True)
            sleep(0.25)
    
    df_similar.to_csv(similar_artists_file, index=False)
    print(f"\nFinished at {datetime.now().strftime('%H:%M:%S')}")

# Usage
if __name__ == "__main__":
    input_file = "data/df_nmf_later.csv"  
    album_art_file = "data/nmf_album_covers.csv"  
    similar_artists_file = "data/nmf_similar_artists.csv"  
    update_album_data(input_file, album_art_file, similar_artists_file, LASTFM_API_KEY)

## Grab the Spotify Link for each New Music Friday Album 🔗

In [ ]:
def generate_spotify_links(batch_size=None, overwrite=False):
    
    output_file = 'data/nmf_album_links.csv'
    
    # Check if the output file already exists
    if os.path.exists(output_file) and not overwrite:
        print(f"Output file {output_file} already exists.")
        print("Loading existing links file instead of regenerating...")
        try:
            existing_links = pd.read_csv(output_file)
            print(f"Loaded {len(existing_links)} existing album links.")
            return existing_links
        except Exception as e:
            print(f"Error loading existing file: {e}")
            print("Continuing with link generation...")
    
    print("Starting Spotify link generation...")
    
    # Initialize Spotify client with your credentials
    client_id = "71faef9605da4db495b691d96a0daa4b"
    client_secret = "832e40da22e049bba93f29d9dbeb2e62"
    
    try:
        print("Authenticating with Spotify...")
        sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
            client_id=client_id,
            client_secret=client_secret
        ))
        print("Spotify authentication successful")
    except Exception as e:
        print(f"Failed to authenticate with Spotify: {e}")
        return None
    
    # Read the NMF data
    try:
        print("Reading NMF data...")
        df = pd.read_csv('data/nmf.csv')
        print(f"Loaded {len(df)} tracks")
        
        # Print the column names to verify
        print("Available columns:", df.columns.tolist())
    except Exception as e:
        print(f"Error reading NMF data: {e}")
        return None
    
    try:
        # Get unique albums
        print("Extracting unique albums...")
        albums = df.drop_duplicates(subset=['Album Name'], keep='first')
        print(f"Found {len(albums)} unique albums")
    except Exception as e:
        print(f"Error extracting albums: {e}")
        return None
    
    # Function to get album ID from track URI with rate limiting
    def get_album_id(track_uri):
        if pd.isna(track_uri):
            print("Warning: Found NaN URI")
            return None
            
        print(f"Processing track: {track_uri}")
        try:
            # Extract just the ID part from the URI (spotify:track:ID_HERE)
            if isinstance(track_uri, str) and 'spotify:track:' in track_uri:
                track_id = track_uri.split(':')[-1]
            else:
                track_id = track_uri  # If it's already an ID
            
            print(f"Extracted track ID: {track_id}")
                
            # Add delay to respect rate limits
            time.sleep(0.1)  # 100ms delay between requests
            track_info = sp.track(track_id)
            album_id = track_info['album']['id']
            print(f"Found album ID: {album_id}")
            return album_id
        except spotipy.exceptions.SpotifyException as e:
            print(f"Spotify API error: {e}")
            if hasattr(e, 'http_status') and e.http_status == 429:  # Too Many Requests
                print("Rate limit hit, waiting longer...")
                time.sleep(5)  # Wait 5 seconds before retrying
                try:
                    track_info = sp.track(track_id)
                    return track_info['album']['id']
                except:
                    print(f"Still failed after retry for track {track_id}")
                    return None
            else:
                print(f"Error getting album ID for track {track_id}: {e}")
                return None
        except Exception as e:
            print(f"Unexpected error for track URI {track_uri}: {e}")
            return None
    
    # Check if we actually have the Track URI column
    track_id_column = None
    for possible_name in ['Track URI', 'track_uri', 'Track ID', 'track_id']:
        if possible_name in albums.columns:
            track_id_column = possible_name
            print(f"Found track identifier column: {track_id_column}")
            break
    
    if track_id_column is None:
        print("ERROR: Could not find a track ID or URI column. Available columns are:")
        print(albums.columns.tolist())
        return None
    
    # Determine which albums to process
    if batch_size is not None:
        albums_to_process = albums.head(batch_size)
        print(f"Processing a batch of {len(albums_to_process)} out of {len(albums)} total albums...")
    else:
        albums_to_process = albums
        print(f"Processing all {len(albums)} albums...")
    
    # Instead of using apply, let's process one by one for better error tracking
    albums['Album ID'] = None
    
    for i, (idx, row) in enumerate(albums_to_process.iterrows()):
        print(f"\nProcessing album {i+1}/{len(albums_to_process)}: {row['Album Name']}")
        album_id = get_album_id(row[track_id_column])
        albums.at[idx, 'Album ID'] = album_id
    
    # Filter out any failed lookups
    processed_albums = albums.dropna(subset=['Album ID'])
    
    processed_albums['Spotify URL'] = 'open.spotify.com/album/' + processed_albums['Album ID'].astype(str)
    
    # Select and reorder columns for output
    output_columns = ['Album Name', 'Artist Name(s)', 'Album ID', 'Spotify URL']
    # Make sure all required columns exist
    output_columns = [col for col in output_columns if col in processed_albums.columns]
    
    output_df = processed_albums[output_columns]
    
    # Save to CSV
    output_df.to_csv(output_file, index=False)
    
    print(f"Successfully generated album links for {len(output_df)} albums")
    return output_df

# Run the function
# Pass overwrite=True to regenerate links even if the file exists
result = generate_spotify_links(overwrite=True)  # Force regenerate all links
print("Function completed")

## Save a HTML copy of this notebook at its newest! 🔽

In [4]:
# Ensure the 'graphics/' directory exists
os.makedirs('graphics', exist_ok=True)

# Load the current notebook with 'utf-8' encoding
notebook_filename = 'Music Taste Machine Learning Data Prep.ipynb'
with open(notebook_filename, 'r', encoding='utf-8') as f:
    notebook_content = nbformat.read(f, as_version=4)

# Export the notebook as HTML
html_exporter = HTMLExporter()
html_data, resources = html_exporter.from_notebook_node(notebook_content)

# Save the HTML to the 'graphics/' folder
output_filename = 'graphics/Music_Taste_Machine_Learning_Data_Prep.html'
with open(output_filename, 'w', encoding='utf-8') as f:
    f.write(html_data)

print(f"HTML version saved to {output_filename}")

HTML version saved to graphics/Music_Taste_Machine_Learning_Data_Prep.html
